In [5]:
import numpy as np
import torch

words = []
with open("google-10000-english-no-swears.txt", 'r') as file:
    words = file.read().splitlines()
    
print(words[:10])

['the', 'of', 'and', 'to', 'a', 'in', 'for', 'is', 'on', 'that']


In [37]:
def getSplitData(words):
    
    xs = []
    ys = []
    
    for word in words:
        word = word.lower()
        word = "{{{"+word+"{"

        for i in range(len(word)-3):
            print(word[i], word[i+1], word[i+2], "-->", word[i+3])
            
            x = [ord(word[i])-97, ord(word[i+1])-97, ord(word[i+2])-97]
            y = [ord(word[i+3])-97]

            xs.append(x)
            ys.append(y)

    # print(xs)
    # print(ys)
    return (xs, ys)
            
data = getSplitData(["ABC", 'xyz', 'cde'])
xs = data[0]
ys = data[1]

{ { { --> a
{ { a --> b
{ a b --> c
a b c --> {
{ { { --> x
{ { x --> y
{ x y --> z
x y z --> {
{ { { --> c
{ { c --> d
{ c d --> e
c d e --> {
